<a href="https://colab.research.google.com/github/Jake-Jasper/AI-Capstone-Project/blob/main/Retrieval_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Retrieval evaluation

Just to evaluate the models using precision and recall...

## Question list (30)
Have the relevant document for each so you can measure, recall, precision etc

Best to complete this after asking the question to the system?

|Doc_ID|Name|Relevant_Q|
|---|---|---|
|1|Evidence & insights - reducing household food waste & plastic packaging (packaged vs loose)|1,2,3,4,5,6,7|
|2|Food waste collection guidance |8, 17|
|3|Food Waste Reduction Roadmap - Executive Summary|9, 16|
|4|Food Waste Reduction Roadmap - Hospitality|9|
|5|Food Waste Reduction Roadmap - Manufacturers|9|
|6|Food Waste Reduction Roadmap - Primary Producers|9,10|
|7|Food Waste Reduction Roadmap - Retail|9,11|
|8|Identifying impacts from food and farm digestates|12,13|
|9|Industry Guidance - Dealing with Household Food Waste at AD Facilities - Management of Liners|14,8, 15|
|10|Literature review - relationship between household food waste collection and food waste prevention|8, 16, 17, 18, 19|
|11|The food waste reduction roadmap progress report 2022|9|
|12|Towards the 2030 Food Waste Commitment -- setting our coalition baseline|14|
|13|Upscaling farm food waste measurement in the UK|22|
|14|WRAP-Evidence Review Plastic Packaging and Fresh Produce 171218|3, 4, 14|

1. "How should I store bananas?"
2. "What temperature should I keep milk?"
3. "How do I increase shelf life of lettuce?
4. "How does selling fresh produce loose, affect waste?"
5. "What is a best-before date for?"
6. "What is a use-by date for?"
7. "How does removing date labels help?"
8. "What are the reasons to not participate in a food waste scheme?"
9. "What is the UN's sustainable development goal with respect to food?"
10. "Which document relates to primary producers and food waste?"
11. "Which document relates to retailers and food waste?"
12. "What is the most common use for digestate?"
13. "What affects the costs of valorisation?"
14. "How many tonnes of food waste are there each year?"
15. "What is the benefit of caddy liners?"
16. "How to prevent food loss?"
17. "How to prevent food waste?"
18. "How does composting affect food waste?"
19. "What are the main drivers of food waste? got to here!"
20. "Online shopping and food waste"
21. "What is involved with Courthold 2030?"
22. "How do you measure in field food waste?"

In [8]:
!pip install -Uq sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 908.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [15]:
import sqlite3
from sentence_transformers import SentenceTransformer, util
import numpy as np

In [102]:
### I think just a dictionary will be able to store the data required

# Question
# documents have been indexed from 0 so document 1 = 0 etc
keys = list(range(1,15))
# answers (relevant questions)
values = [
    [1,2,3,4,5,6,7],
    [8,17],
    [9,16],
    [9],
    [9],
    [9,10],
    [9,11],
    [12,13],
    [14,8,15],
    [8,16,17,18,19],
    [9],
    [14],
    [22],
    [1,3,4,14],

]

QA = dict(zip(keys,values))

In [11]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
# Establish a connection to the SQLite database
conn = sqlite3.connect('/content/drive/MyDrive/Knowledge-database-6-4-24.db')

# Create a cursor object
cursor = conn.cursor()

cursor.execute("""SELECT * FROM documents LIMIT 5""")

for row in cursor:
    print(row)

conn.close()

(1, 0, '01 - Evidence & insights - reducing household food waste & plastic packaging (packaged vs loose).pdf', 'EVIDENCE AND INSIGHTSReducing household food waste and plastic packagingGround-breaking new research into the\xa0factors affecting how much uncut fresh-produce goes to waste in the home.\x0c', b'\xca\xa4\xd7<\x0b\x8d\x1c\xbc\x96U\x97=\x0f\x0e\xe2<+\xc7\xbb=s\xea\x18\xbc\xcbe\x8f;m\xa7\x1e<D\xea\x8f\xbdx\\f;\xc9M\xca<\xa9\xb9/\xbdF\xd8\xb4\xbc\xdf\n2;hu\xfe<\xd05\x9b\xbd\xbd\xf4[=A\xd4\x83<\xc6\x12\xae\xbdt\xd6\xab\xbc\xf0\xe0\x94=\x08\xa2\xf1\xbcrl\x8b=\xdf\x8e6\xbc\x82oo=\xa1l\x12=\xdb\xe8\x91<\x97\x10\x88\xbdWx\t\xbd\xa8\x93\x9a\xbd\xa6K?=_\xf7\x19=\x10o\x9c=>j\xe3:\x1e>p=\x14\x1e<=Ey\x9a=\xd6f\xee\xbb\xeb\x8a\x90=\xfaC4\xbb{?=\xbc\x9d\xa3\xdd\xbcP\xad/=\x17\x8d?\xba\x81\xb8\xf3\xbc\xe4\xf1\x86\xbc@\xe6\xb4\xbd\x9c\xf0\xa9\xbdn\xe76\xbd\xa0\xc0\x9a\xbd\xf0cQ=6\xf6\xc1=\xcfp\xa4\xbb\x9d&\xdc\xbc\xf8#k=\xe0\xb4\xeb\xbc\xe2/\xca=\x1a%\xc6\xbc)\x87"\xbd\x01\xa0\xc8<J\x88\x96=\x

In [97]:
def score_retrieval(question=["No question"],
                    model=None, db='/content/drive/MyDrive/Knowledge-database-6-4-24.db',
                    test=True):
    test = test
    relevant = []
    q = model.encode(question)
    conn = sqlite3.connect(db)

    # Create a cursor object
    cursor = conn.cursor()
    # load row by row
    cursor.execute('SELECT * FROM documents')

    for row in cursor:
        score  = util.pytorch_cos_sim(q, np.frombuffer(row[4], dtype=np.float32)).numpy()
        # if meets the score threshold and is not already in the list
        if score >= 0.4 and row[1] not in relevant:
            # if high score consider it
            if test == True:
                relevant.append((row[1] + 1, row[3]))
            else:
                relevant.append((row[1] + 1))


    return list(set(relevant))

In [126]:
# What is the UN's sustainable development goal with respect to food?
retrieved_docs = score_retrieval(question =["Which document relates to retailers and food waste?"], model=model, test=False)
retrieved_docs

[1, 3, 5, 7, 9, 11, 13]

In [101]:
QA

{1: [1, 2, 3, 4, 5, 6, 7],
 2: [8, 17],
 3: [9, 16],
 4: [9],
 5: [9],
 6: [9, 10],
 7: [9, 11],
 8: [12, 13],
 9: [14, 8, 15],
 10: [8, 16, 17, 18, 19],
 11: [9],
 12: [14],
 13: [22],
 14: [3, 4, 14]}

In [127]:
## get scores for the document retrieval

def scoring(q, qa=QA):
    q_no = q
    # true positive
    all_relevant = []
    for k, v in QA.items():
        if q_no in v:
            all_relevant.append(k)
    correct = 0
    for i in retrieved_docs:
        if i in all_relevant:
            correct +=1
    # precision
    if len(retrieved_docs) == 0:
        precision , recall, f1 = 0,0,0
    else:
        precision = abs(len(set(all_relevant) & set(retrieved_docs))) / abs(len(retrieved_docs))
        # recall
        recall = abs(len(set(all_relevant) & set(retrieved_docs))) / abs(len(all_relevant))
        if precision == 0 or recall == 0:
            f1 = 0.0
        #F1
        else:
            f1 = (2*precision*recall)/(precision + recall)

    return (precision, recall, f1)

scoring(11)

(0.14285714285714285, 1.0, 0.25)